In [1]:
import json
from imslp_scraping import get_all_composer_pieces, get_composer_url
from pieces import create_piece
from ydata_profiling import ProfileReport
from typing import List, Dict, Any
import pyarrow
import datetime
import pandas as pd
import numpy as np
import polars as pl


def piece_to_dict(piece):
    piece_dict = vars(piece).copy()
    piece_dict["movements"] = json.dumps([vars(m) for m in piece.movements])
    return piece_dict

/Users/michaelbrandt/personal/PracticeLogger/data_ingest/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
composers = [
    "Bach, Johann Sebastian",
    "Mozart, Wolfgang Amadeus",
    "Prokofiev, Sergey",
    "Liszt, Franz",
    "Debussy, Claude",
    "Tchaikovsky, Pyotr",
    "Schumann, Robert",
    "Shostakovich, Dmitry",
    "Mendelssohn, Felix",
    "Schubert, Franz",
    "Dvořák, Antonín",
    "Grieg, Edvard",
    "Ravel, Maurice",
    "Mahler, Gustav",
    "Ginastera, Alberto",
    "Mussorgsky, Modest",
    "Sibelius, Jean",
    "Haydn, Joseph",
    "Saint-Saëns, Camille",
    "Satie, Erik",
    "Handel, George Frideric",
    "Scarlatti, Domenico",
    "Vivaldi, Antonio",
    "Clementi, Muzio",
    "Couperin, François",
    "Brahms, Johannes",
    "Schumann, Clara",
    "Franck, César",
    "Wagner, Richard",
    "Verdi, Giuseppe",
    "Bruckner, Anton",
    "Berlioz, Hector",
    "Albéniz, Isaac",
    "Fauré, Gabriel",
    "Strauss, Richard",
    "Bartók, Béla",
    "Stravinsky, Igor",
    "Gershwin, George",
    "Poulenc, Francis",
    "Copland, Aaron",
    "Granados, Enrique",
    "Medtner, Nikolay",
    "Bach, Carl Philipp Emanuel",
    "Telemann, Georg Philipp",
    "Weber, Carl Maria von",
    "Hummel, Johann Nepomuk",
    "Paganini, Niccolò",
    "Field, John",
    "Balakirev, Mily",
    "Borodin, Aleksandr",
    "Rimsky-Korsakov, Nikolay",
    "Glazunov, Aleksandr",
    "Chausson, Ernest",
    "Smetana, Bedřich",
    "Bizet, Georges",
    "Offenbach, Jacques",
    "Massenet, Jules",
    "Busoni, Ferruccio",
    "Ives, Charles",
    "Schoenberg, Arnold",
    "Berg, Alban",
    "Milhaud, Darius",
    "Hindemith, Paul",
    "Vaughan Williams, Ralph",
    "Elgar, Edward",
    "Britten, Benjamin",
    "Janáček, Leoš",
    "Nielsen, Carl",
    "Bernstein, Leonard",
    "Glinka, Mikhail",
    "Godowsky, Leopold",
    "Lully, Jean-Baptiste",
    "Ligeti, György",
    "Webern, Anton",
    "Beethoven, Ludwig van",
    "Rachmaninoff, Sergei",
    "Chopin, Frédéric",
    "Scriabin, Aleksandr",
]
# composers = ["Rachmaninoff, Sergei", ""Villa-Lobos, Heitor""]

pieces = []
for composer in composers:
    print(f"{datetime.datetime.now().time()} Starting import for {composer}")
    url = get_composer_url(composer)
    data = get_all_composer_pieces(url)
    # print("data", data)
    for piece_url in data:
        if len(pieces) % 100 == 0:
            print(f"{datetime.datetime.now().time()} pieces count: {len(pieces)}")
        try:
            piece = create_piece(url=piece_url)
        except ValueError as e:
            print(e, piece_url)
            next
        # print(piece)
        pieces.append(piece)

20:25:14.959293 Starting import for Bach, Johann Sebastian


KeyboardInterrupt: 

In [5]:
pieces_dict = [piece_to_dict(piece) for piece in pieces]
pl.Config.set_fmt_str_lengths(50)
pl.Config.set_tbl_rows(10)
df = pl.DataFrame(pieces_dict, strict=False, infer_schema_length=1000)
# print(df)
df.head(100)
df.write_parquet("full_df.parquet")

In [3]:
file = Path("full_df.parquet")
df = pl.read_parquet(file)

NameError: name 'Path' is not defined

In [7]:
# get rid of composers that have under 10 works
filtered_df = (
    df.group_by("composer_name")
    .agg(pl.count("work_name").alias("count"))
    .filter(pl.col("count") > 10)
    .join(df, on="composer_name")
)

In [8]:
pl.Config.set_fmt_str_lengths(50)
pl.Config.set_tbl_rows(50)
filtered_df = filtered_df.filter(
    ~(
        #    (pl.col('instrumentation').list.len() == 1) &
        pl.col("instrumentation").list.get(0).str.contains("chorus|SATB|mixed chorus")
    )
)
# filtered_df.select('instrumentation')

In [ ]:
instrument_mapping = {
    "viol": "violin",
    "piano)": "piano",
    "(piano": "piano",
    "Piano Solo": "piano",
    "1 piano": "piano",
    "piano (no.12 = 2 voices": "piano",
    "piano (or harp (no.7 only)": "piano",
    "piano (nos.5-7)": "piano",
    "piano (arranged)": "piano",
    "2 horns)": "2 horns",
}

filtered_df = filtered_df.with_columns(
    pl.col("instrumentation")
    .list.eval(
        pl.when(pl.element().str.to_lowercase().is_in(instrument_mapping.keys()))
        .then(pl.element().str.to_lowercase().replace(instrument_mapping))
        .otherwise(pl.element().str.to_lowercase())
    )
    .alias("instrumentation")
)
unique_instruments = (
    filtered_df.select("instrumentation", "work_name", "composer_name", "imslp_url")
    .explode("instrumentation")
    .unique()
    .sort(by="work_name")
)
# problematic instrumentation: https://imslp.org/wiki/10_Pieces_for_Piano,_Op.12_(Prokofiev,_Sergey)
# https://imslp.org/wiki/Ebbe_Skammels%C3%B8n%2C_CNW_21_(Nielsen%2C_Carl)
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_tbl_rows(1000)
# unique_instruments.head(10000)

polars.config.Config

composer_name,count,work_name,catalogue_desc_str,catalogue_type,catalogue_number,catalogue_number_secondary,catalogue_id,composition_year,composition_year_string,key_signature,movements,sub_piece_type,sub_piece_count,instrumentation,nickname,piece_style,imslp_url,wikipedia_url
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
13160,13160,13160,11579,11579,11240,666,0,9386,9498,8654,13160,8161,8161,13160,6496,13160,13160,3381


In [ ]:
df = filtered_df.unique()
from supabase_database import SupabaseDatabase

db = SupabaseDatabase()
try:
    successful, failed = db.bulk_insert_from_df(df)
    print(f"Successfully inserted {successful} pieces")
    if failed:
        print("Failed inserts:")
        for work_name, error in failed:
            print(f"- {work_name}: {error}")
finally:
    db.close()

Successfully inserted 13167 pieces
Failed inserts:
- Weimarer (Gothäer) Passion: invalid input value for enum catalogue_type: "bc"
LINE 9: ...       1, 'Weimarer (Gothäer) Passion', 'BC D 1', 'bc', NULL...
                                                             ^

- Laß, Fürstin, laß noch einen Strahl: value too long for type character varying(255)

- Fantasia: invalid input value for enum catalogue_type: "kv"
LINE 9:             6, 'Fantasia', 'KV 616a', 'kv', 616, NULL, NULL,...
                                              ^

- Horn Concerto: invalid input value for enum catalogue_type: "kanh"
LINE 9: ...      6, 'Horn Concerto', 'K²Anh.96a. ; K⁶.494a', 'kanh', 96...
                                                             ^

- Allegro: invalid input value for enum catalogue_type: "kv:"
LINE 9:             6, 'Allegro', 'KV6: 484e', 'kv:', 484, NULL, NUL...
                                               ^

- Sonatensatz: invalid input value for enum catalogue_type: "kv"
LIN

In [6]:
profile = ProfileReport(df.to_pandas(), title="Pandas Profiling Report")

profile.to_file("data_summary.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/Users/michaelbrandt/personal/PracticeLogger/data_ingest/.venv/lib/python3.12/site-packages/ydata_profiling/model/pandas/summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 310.28it/s]
